# Sentinel 2 Atmospheric Correction in Google Earth Engine

### Import modules 
and initialize Earth Engine

In [1]:
# https://github.com/samsammurphy/gee-atmcorr-S2

import ee
import geemap
#from Py6S import * #The Py6S package is embedded into the atmospheric functions
import os, sys, time, math, datetime
from atmospheric import atmCorr, ConvS2Collection
#from atmospheric import Atmospheric, Conversion

ee.Initialize()

In [ ]:
# Test SixS from Py6S package
#s = SixS()

#SixS.test()

# Update geemap package
#geemap.update_package()

### time and place
Define the time.

In [ ]:
# Define a year to calculate.
# year1 should be between 2016 to 2019 for our case of study.
# Create year1 = 2020 to apply Machine Learning model.
year1 = 2020
year2 = year1 + 1

start = ee.Date(str(year1) + '-10-01')
stop = ee.Date(str(year2) + '-01-01')

# Percentage of cloud coverage only fot he year 2020 is 20% the other years are 40%
percentage = 20

### Region of study
The following code will determine the region where the analysis is apply

In [ ]:
# Rectangle with the ROI
geom = ee.Geometry.Rectangle(-70.8200199999999995, -33.7983520000000013, 
                             -70.8068049999999971, -33.7804399999999987)

### Filter the Sentinel 2 collection
The following code will grab the ImageCollection on the dates and ROI.

In [ ]:
S2 = ee.ImageCollection('COPERNICUS/S2')\
     .filterBounds(geom)\
     .filterDate(start, stop)\
     .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', percentage)\
     .sort('system:time_start')\
     .distinct('system:time_start')

# GEE path to save the image inside of a image collection (GEE account)
gpath = 'users/diegoalarcondiaz/tester/'

# Conversion function in a bash process
ConvS2Collection.conversion(geom, S2, gpath)

### Check image Sentinel-2

Here we can check the beginning of the Sentinel-2 image collection TOA.

In [ ]:
# Create a default map
Map = geemap.Map(center=[-70.81, -33.789], zoom=15)

vis_params = {"min": 300,
              "max": 3500,
              "bands": ["B4", "B3", "B2"]}

S2_TOA = ee.ImageCollection('COPERNICUS/S2')\
           .filterBounds(geom)\
           .filterDate(start, stop)\
           .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', percentage)

Map.centerObject(geom)
Map.addLayer(S2_TOA, {}, "Sentinel-2 TOA", False)
Map.add_time_slider(S2_TOA, vis_params)
Map.addLayer(geom)
Map

In [ ]:
# Create a default map
Map = geemap.Map(center=[-70.81, -33.789], zoom=15)

vis_params = {"min": 0,
              "max": 0.3,
              "bands": ["B4", "B3", "B2"]}

S2_BOA = ee.ImageCollection('users/diegoalarcondiaz/sen2_cor') \
             .filterBounds(geom) \
             .filterDate(start, stop)

Map.centerObject(geom)
Map.addLayer(S2_BOA, {}, "Sentinel-2 BOA", False)
Map.add_time_slider(S2_BOA, vis_params)
Map.addLayer(geom)
Map